# Ensemble Learning

## Initial Imports

In [149]:
import warnings
warnings.filterwarnings('ignore')

In [150]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [151]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier

## Read the CSV and Perform Basic Data Cleaning

In [152]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [153]:
# Fitting and encoding the columns with the LabelEncoder
le = LabelEncoder()
# Encoding home_ownership column
le.fit(df["home_ownership"])
df["home_ownership"] = le.transform(df["home_ownership"])

# Encoding verification_status column
le.fit(df["verification_status"])
df["verification_status"] = le.transform(df["verification_status"])

# Encoding loan_status column
le.fit(df["loan_status"])
df["loan_status"] = le.transform(df["loan_status"])

le.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = le.transform(df["debt_settlement_flag"])

le.fit(df["hardship_flag"])
df["hardship_flag"] = le.transform(df["hardship_flag"])

le.fit(df["issue_d"])
df["issue_d"] = le.transform(df["issue_d"])

le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])
le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])
le.fit(df["next_pymnt_d"])
df["next_pymnt_d"] = le.transform(df["next_pymnt_d"])
le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])
type(df.columns)

pandas.core.indexes.base.Index

In [154]:
# Create our features
X = df.drop(columns="loan_status")
# Create our target
y = df["loan_status"]

In [155]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [156]:
# Check the balance of our target values
pd.DataFrame(df["loan_status"]).value_counts()

loan_status
1              68470
0                347
dtype: int64

In [157]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=76)
X_train.shape

(51612, 85)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [158]:
# Create the StandardScaler instance
scaler = StandardScaler()


In [159]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
x_scaler = scaler.fit(X_train)

In [160]:
# Scale the training and testing data
X_train_scaled = x_scaler.transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [161]:
# Resample the training data with the BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [162]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)


In [163]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [164]:
# Print the imbalanced classification report
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Balanced Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,66,29
Actual 1,1586,15524


Balanced Accuracy Score : 0.8010212556522809
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.04      0.69      0.91      0.08      0.79      0.62        95
          1       1.00      0.91      0.69      0.95      0.79      0.64     17110

avg / total       0.99      0.91      0.70      0.95      0.79      0.64     17205



In [165]:
# List the features sorted in descending order by feature importance
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)


[(0.07285536670686354, 'total_rec_prncp'),
 (0.06476585685219305, 'last_pymnt_amnt'),
 (0.0601860018574273, 'total_pymnt'),
 (0.059196666294636484, 'total_pymnt_inv'),
 (0.059082427179768734, 'total_rec_int'),
 (0.0265452311326471, 'int_rate'),
 (0.025109532800851386, 'issue_d'),
 (0.018107325458682447, 'dti'),
 (0.017818317824747014, 'installment'),
 (0.015927524731339918, 'mo_sin_old_rev_tl_op'),
 (0.015874219935018266, 'annual_inc'),
 (0.015871641853336685, 'out_prncp_inv'),
 (0.01585827386163018, 'out_prncp'),
 (0.015594533581666696, 'max_bal_bc'),
 (0.015500618527899338, 'mths_since_recent_inq'),
 (0.01526177981379713, 'mo_sin_old_il_acct'),
 (0.015254179951084109, 'revol_bal'),
 (0.014978035435220867, 'tot_hi_cred_lim'),
 (0.014953871443243064, 'bc_util'),
 (0.014659637987985806, 'tot_cur_bal'),
 (0.014194486295041712, 'mths_since_recent_bc'),
 (0.013804433307104654, 'avg_cur_bal'),
 (0.013491936327265228, 'total_bal_il'),
 (0.01338873172247855, 'total_rev_hi_lim'),
 (0.013349028

### Easy Ensemble Classifier

In [166]:
# Train the Classifier
rf_model = EasyEnsembleClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [167]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)

In [168]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [169]:

# Print the imbalanced classification report
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Balanced Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,87,8
Actual 1,975,16135


Balanced Accuracy Score : 0.9294026269648406
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.08      0.92      0.94      0.15      0.93      0.86        95
          1       1.00      0.94      0.92      0.97      0.93      0.87     17110

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Easy Ensemble Classifier

2. Which model had the best recall score?

    Easy Ensemble Classifier.

3. Which model had the best geometric mean score?

    Easy Ensemble Classifier.

4. What are the top three features?

   total_rec_prncp,last_pymnt_amnt,'total_pymnt'